# Hyperparameter optimization - ResNet50
* This notebook was used to optimize hyperparameters for ResNet50 architecture, using Keras tuner library
* It already contains the results from the hyperband search

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Google Colab does not have tensorflow_addons installed by default
!pip install tensorflow-addons
from tensorflow_addons.metrics import CohenKappa

# Install keras-tuner
!pip install keras-tuner --upgrade
import keras_tuner as kt

# FILEPATHS

In [2]:
DRIVE_DIR = '/content/drive/MyDrive/MURA/' # Directory in my personal Google Drive with all source files and datasets
MURA_DIR = '/content/original/' # Directory with original MURA dataset
CLAHE_2_DIR = '/content/clahe_2/' # Directory for CLAHE preprocessed dataset with clipLimit=2
CLAHE_10_DIR = '/content/clahe_10/' # Directory for CLAHE preprocessed dataset with clipLimit=10
CROPPED_DIR = '/content/cropped/' # Directory for custom cropping preprocessed dataset
DATAFRAME_PATH = DRIVE_DIR + '/tvt_detailed_paths.csv' # Path to csv file with dataset information (train-valid-test split)

# Google Colab specific section

## Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Google colab functions

Next cell containts function definitions, that are needed or useful when running notebook in Google Colab, such as copying dataset from Google Drive, checking GPU utilization, copying other files from Google Drive.

In [4]:
# This function is from Google Colab guide notebooks: https://colab.research.google.com/notebooks/pro.ipynb
def check_gpu():
  """
  Prints information about available GPU or message saying there isn't one
  """
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)


def init_file_struct():
  """
  Prep file structure in Google Colab and copies neccessary files from Google Drive
  """
  !cp "{DRIVE_DIR}src/utils.py" .
  !cp "{DRIVE_DIR}src/image_preprocessing.py" .


def copy_dataset(filename):
  """
  Copies and unzips dataset from Google Drive and deletes the zipped file

  Parameters
  ----------
  filename : str
    Dataset filename, without its path, the path will be added from constants
  """
  !cp {DRIVE_DIR}{filename} /content/

  !unzip -q /content/{filename}

  !rm /content/{filename}

## Import python files after copying them from Google Colab

In [5]:
init_file_struct()
from utils import *
from image_preprocessing import *

## Copy desired dataset to be used

In [6]:
copy_dataset('original.zip')
# copy_dataset('clahe_2.zip')
# copy_dataset('clahe_10.zip')
# copy_dataset('cropped.zip')

# SETUP

In [7]:
# DATAFRAME
BODY_PART = 'SHOULDER' # Which body part to use, can be "ALL" for full dataset
# IMAGE AUGMENTATION
ROTATION_R = 20 # Rotation range
W_SHIFT_R = 0.05 # Width shift range
H_SHIFT_R = 0.05 # Height shift range
BRIGHTNESS_R = (0.9, 1.1) # Brightness range
ZOOM_R = 0.1 # Zoom range
H_FLIP = True # Flip the image horizontally
PRE_FUNC = rescale # Preprocessing function
# DATAFRAME FLOW
IMAGE_SIZE = (224, 224) # Resize all images to this size
# MODEL
BASE_NAME = 'ResNet50' # Name corresponding to one of the architectures from Keras functional API
WEIGHTS = 'imagenet' # ImageNet pre-trained weights or a path to stored model weights
INPUT_SHAPE = (224, 224, 3) # Model input shape
MODEL_NAME = BASE_NAME + '_tuning' # Name used for storing model weights during and after training
# TUNING
BATCH_SIZE = [8, 16, 32] # Batch size options
POOLING = ['avg', 'max'] # Pooling options
LEARNING_RATE = [0.001, 0.0005, 0.00025, 0.0001, 0.00005] # Adam learning rate options
MAX_EPOCHS = 7 # Maximum epochs for one trial run
EPOCHS = 80 # Maximum number of epochs for whole tuning

# Create datagens and flows

In [8]:
# Create dataframes from train valid split
train_df = get_dataframe(body_part=BODY_PART, split='train', path=DATAFRAME_PATH)
valid_df = get_dataframe(body_part=BODY_PART, split='valid', path=DATAFRAME_PATH)

# Create ImageDataGenerators, train_gen uses specified online augmentation, valid_gen only preprocesses images
train_gen, valid_gen = create_generators(rotation_r=ROTATION_R,
                                         w_shift_r=W_SHIFT_R,
                                         h_shift_r=H_SHIFT_R,
                                         brightness_r=BRIGHTNESS_R,
                                         zoom_r=ZOOM_R,
                                         h_flip=H_FLIP,
                                         pre_func=PRE_FUNC)

# Create dataframe flows, using batch size as hyperparameter
train_flow, valid_flow = create_dataframe_flows(train_gen=train_gen,
                                                valid_gen=valid_gen,
                                                train_df=train_df,
                                                valid_df=valid_df,
                                                directory=MURA_DIR,
                                                img_size=IMAGE_SIZE)

Found 8257 validated image filenames belonging to 2 classes.
Found 563 validated image filenames belonging to 2 classes.


In [9]:
# Function for building the model using tuned hyperparameters
def build_hyperparam(hp):
  hp_pool = hp.Choice('pool', POOLING) # Pooling after last conv layer
  hp_lr = hp.Choice('lr', LEARNING_RATE) # Adam initial learning rate
  hp_batch_size = hp.Choice('batch_size', BATCH_SIZE) # Batch size

  # Set batch size
  train_flow.batch_size = hp_batch_size
  valid_flow.batch_size = hp_batch_size

  # Call function for building model from utils.py
  model = build_model(base_name=BASE_NAME,
                      weights=WEIGHTS,
                      shape=INPUT_SHAPE,
                      name=MODEL_NAME,
                      pooling=hp_pool,
                      optimizer=Adam(learning_rate=hp_lr))
  return model

In [10]:
# Create tuner using Hyperband search
tuner = kt.Hyperband(
    hypermodel=build_hyperparam,
    objective=kt.Objective('val_kappa', 'max'),
    max_epochs=MAX_EPOCHS,
    overwrite=True,
    directory=DRIVE_DIR + "tuning",
    project_name="resnet50_hp"
)

94781440/94765736 [==============================] - 0s 0us/step


In [11]:
early_stop = EarlyStopping(monitor='val_kappa',
                           mode='max',
                           min_delta=0,
                           patience=5,
                           restore_best_weights=True)

tuner.search(x=train_flow, epochs=EPOCHS, validation_data=valid_flow, class_weight=get_class_weights(train_df), verbose=1, callbacks=[early_stop])

Trial 10 Complete [00h 20m 57s]
val_kappa: 0.33763033151626587

Best val_kappa So Far: 0.5415279865264893
Total elapsed time: 02h 14m 08s
INFO:tensorflow:Oracle triggered exit


In [54]:
# Get best model parameters
best_hp = tuner.get_best_hyperparameters(1)[0]

# Get search space and best parameters
hp_config = best_hp.get_config().get('space')
hp_params = best_hp.get_config().get('values')

# Print summary
print('------------------------------\nHyperparameter tuning summary:\n------------------------------')
print('Model:', BASE_NAME)
print('Searched parameters:\n------------------------------')
for parameter in hp_config:
  print(parameter['config']['name'], ':', parameter['config']['values'])
print('------------------------------')
print('Best parameter config:\n------------------------------')
for key in hp_params.keys():
  print(key, ':', hp_params[key])
print('------------------------------\n')

------------------------------
Hyperparameter tuning summary:
------------------------------
Model: ResNet50
Searched parameters:
------------------------------
pool : ['avg', 'max']
lr : [0.001, 0.0005, 0.00025, 0.0001, 5e-05]
batch_size : [8, 16, 32]
------------------------------
Best parameter config:
------------------------------
pool : avg
lr : 0.0001
batch_size : 32
tuner/epochs : 7
tuner/initial_epoch : 0
tuner/bracket : 0
tuner/round : 0
------------------------------



In [12]:
# Optional - print full summary
tuner.results_summary()

Results summary
Results in /content/drive/MyDrive/MURA/tuning/resnet50_hp
Showing 10 best trials
Trial summary
Hyperparameters:
pool: avg
lr: 0.0001
batch_size: 32
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.5415279865264893
Trial summary
Hyperparameters:
pool: avg
lr: 5e-05
batch_size: 16
tuner/epochs: 7
tuner/initial_epoch: 3
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0003
Score: 0.5300976037979126
Trial summary
Hyperparameters:
pool: avg
lr: 5e-05
batch_size: 16
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.5173535346984863
Trial summary
Hyperparameters:
pool: max
lr: 0.00025
batch_size: 8
tuner/epochs: 7
tuner/initial_epoch: 3
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0002
Score: 0.40029674768447876
Trial summary
Hyperparameters:
pool: max
lr: 0.00025
batch_size: 8
tuner/epochs: 3
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.39198946952819824
Trial summary
Hyperparameters:
pool: avg
